In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../archive/seg_train/'
test_path = '../../blur/data/blur_single/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size=64,
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            batch_size=64,
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 21:29:25.093607: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 21:29:25.093746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch=len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-29 21:29:25.593228: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 21:29:26.581640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


220/220 [==============================] - ETA: 0s - loss: 1.6260 - accuracy: 0.3901

2022-08-29 21:29:51.521985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


220/220 [==============================] - 28s 121ms/step - loss: 1.6260 - accuracy: 0.3901 - val_loss: 1.8551 - val_accuracy: 0.2363
Epoch 2/10
220/220 [==============================] - 26s 117ms/step - loss: 1.1987 - accuracy: 0.5268 - val_loss: 1.2205 - val_accuracy: 0.4983
Epoch 3/10
220/220 [==============================] - 26s 117ms/step - loss: 1.0764 - accuracy: 0.5807 - val_loss: 1.1939 - val_accuracy: 0.5097
Epoch 4/10
220/220 [==============================] - 26s 119ms/step - loss: 0.9972 - accuracy: 0.6212 - val_loss: 1.6923 - val_accuracy: 0.4093
Epoch 5/10
220/220 [==============================] - 26s 117ms/step - loss: 0.9305 - accuracy: 0.6543 - val_loss: 1.2931 - val_accuracy: 0.4943
Epoch 6/10
220/220 [==============================] - 26s 117ms/step - loss: 0.8901 - accuracy: 0.6715 - val_loss: 1.2580 - val_accuracy: 0.5167
Epoch 7/10
220/220 [==============================] - 26s 118ms/step - loss: 0.8436 - accuracy: 0.6930 - val_loss: 1.3474 - val_accuracy: 0.5

In [11]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

47/47 - 2s - loss: 1.4610 - accuracy: 0.4820 - 2s/epoch - 43ms/step
Accuracy of the model on the test set:  0.4819999933242798
